# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+5) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 14.10.24 23:59__   
__Жесткий дедлайн: 17.10.24 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('biotech_news.tsv', sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


## Предобработка лейблов


__Задание 1 (1.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [2]:
fixed_label_order = [
    'event organization', 'executive statement', 'regulatory approval', 'hiring',
    'foundation', 'closing', 'partnerships & alliances', 'expanding industry',
    'new initiatives or programs', 'm&a', 'service & product providing', 'event organisation',
    'new initiatives & programs', 'subsidiary establishment', 'product launching & presentation',
    'product updates', 'executive appointment', 'alliance & partnership', 'ipo exit',
    'article publication', 'clinical trial sponsorship', 'company description',
    'investment in public company', 'other', 'expanding geography', 'participation in an event',
    'support & philanthropy', 'department establishment', 'funding round', 'patent publication'
    ]
label_to_index = {label: idx for idx, label in enumerate(fixed_label_order)}

In [3]:
def encode_labels(labels, label_to_index):
    if pd.isna(labels):
        return np.zeros(len(label_to_index), dtype=int)

    vector = np.zeros(len(label_to_index), dtype=int)
    for label in labels.split(', '):
        if label in label_to_index:
            vector[label_to_index[label]] = 1
    return vector

In [4]:
dataset['label_vector'] = dataset['labels'].apply(lambda x: encode_labels(x, label_to_index))

# Проверяем результат
print("Уникальные классы:", fixed_label_order)
print("Пример закодированных меток:")
print(dataset[['labels', 'label_vector']].head())

Уникальные классы: ['event organization', 'executive statement', 'regulatory approval', 'hiring', 'foundation', 'closing', 'partnerships & alliances', 'expanding industry', 'new initiatives or programs', 'm&a', 'service & product providing', 'event organisation', 'new initiatives & programs', 'subsidiary establishment', 'product launching & presentation', 'product updates', 'executive appointment', 'alliance & partnership', 'ipo exit', 'article publication', 'clinical trial sponsorship', 'company description', 'investment in public company', 'other', 'expanding geography', 'participation in an event', 'support & philanthropy', 'department establishment', 'funding round', 'patent publication']
Пример закодированных меток:
  labels                                       label_vector
0  other  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1  other  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2  other  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3  other  [0, 0, 0, 0, 0, 0, 

In [5]:
dataset

,text,labels,label_vector
0,drive your plow over the bones of the dead by ...,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,in the recently tabled national budget denel h...,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,shares take a break its good for you picture g...,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,reso is currently hiring for two positions pro...,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,charter buyer club what is the charter buyer c...,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
3034,published less than an hour ago a grateful fam...,"funding round, support & philanthropy, executi...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3035,a cenexelcenter of excellence joined nearly 10...,clinical trial sponsorship,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3036,jun 29 2020 8 47 a m pt reply in response to t...,"new initiatives or programs, funding round, ex...","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3037,whatsapp photo supplied red river waste soluti...,"service & product providing, closing, company ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [6]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(
    dataset['text'],
    dataset['label_vector'].tolist(),
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1.5 балла)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
import re
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
import re

In [89]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [90]:
texts_train_cleaned = texts_train.apply(clean_text)

vectorizer = CountVectorizer(stop_words=stop_words)
X_train_counts = vectorizer.fit_transform(texts_train_cleaned)

word_counts = X_train_counts.toarray().sum(axis=0)
word_to_index = vectorizer.vocabulary_
index_to_word = {idx: word for word, idx in word_to_index.items()}

total_docs = len(texts_train_cleaned)
word_frequencies = {index_to_word[idx]: count / total_docs for idx, count in enumerate(word_counts)}

min_freq = 0.01
max_freq = 0.95
filtered_words = {word for word, freq in word_frequencies.items() if min_freq <= freq <= max_freq}


In [91]:
texts_train_cleaned = texts_train.apply(clean_text)
texts_test_cleaned = texts_test.apply(clean_text)

print("Пример очищенного текста:")
print(texts_train_cleaned.iloc[0])

Пример очищенного текста:
alex nitkin waterton ceo david schwartz left pathway living ceo jerome finis pathway living facility oak hill credit pathway living real estate investment firm waterton bought majority stake pathway living chicago based senior living operator facilities totaling units waterton bought percent share half seats board pathway living waterton ceo david schwartz told real deal chicago based investor last month paid undisclosed sum boost share percent four years waterton shared ownership housing operator helped finance development acquisition nine new facilities including million pickup grandbrier northwest suburban prospect heights last year pathway another three residences development pipeline including one suburban la grange westmont third planned philadelphia area marking first step east coast expansion push waterton help power pathway living offers mix independent living assisted living memory care facilities average resident age companys expansion put prime pos

__Задание 3 (2 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +5 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [92]:
from gensim.corpora.dictionary import Dictionary
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

In [93]:
def word_tokenizer(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = text.split()
    return tokens

In [94]:
texts_train_tokenized = texts_train_cleaned.apply(word_tokenizer)
texts_test_tokenized = texts_test_cleaned.apply(word_tokenizer)

In [95]:
dictionary = Dictionary(texts_train_tokenized)
dictionary.add_documents(([['PAD', 'UNK']]))
print("Размер словаря:", len(dictionary))

Размер словаря: 39368


In [96]:
def collate_fn(batch):
    texts, labels = zip(*batch)
    pad_token_id = dictionary.token2id['PAD']
    unk_token_id = dictionary.token2id['UNK']
    input = [torch.tensor(dictionary.doc2idx(text, unknown_word_index=dictionary.token2id['UNK']), dtype=torch.long) for text in texts]
    return (pad_sequence(input, batch_first=True, padding_value=pad_token_id).long(), torch.tensor(labels).float())

Даталоадеры

In [97]:
train_loader = DataLoader(list(zip(texts_train_tokenized, y_train)), collate_fn=collate_fn, shuffle=True, batch_size=32)

test_loader = DataLoader(list(zip(texts_test_tokenized, y_test)), collate_fn=collate_fn, shuffle=False, batch_size=32)

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [19]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс.

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, num_classes, pad_token_id, embedding_dim, hidden_dim):
        super(RNN, self).__init__()

        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_dim = hidden_dim
        self.hidden_transform = nn.Linear(embedding_dim + hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, num_classes)
        self.pad_token_id = pad_token_id

    def forward(self, input_tokens, initial_hidden_state=None):
        batch_size, seq_length = input_tokens.shape

        embedded_tokens = self.embedding_layer(input_tokens)

        if initial_hidden_state is None:
            hidden_state_t = torch.zeros(batch_size, self.hidden_dim).to(input_tokens.device)
        else:
            hidden_state_t = initial_hidden_state

        hidden_state_sequence = []

        for timestep in range(seq_length):
            token_embedding_t = embedded_tokens[:, timestep, :]
            concatenated_state = torch.cat((token_embedding_t, hidden_state_t), dim=1)
            hidden_state_t = torch.tanh(self.hidden_transform(concatenated_state))
            hidden_state_sequence.append(hidden_state_t.unsqueeze(1))


        hidden_state_sequence = torch.cat(hidden_state_sequence, dim=1)
        sequence_lengths = (input_tokens == self.pad_token_id).int().argmax(dim=-1) - 1
        sequence_lengths = sequence_lengths % input_tokens.shape[1]
        final_hidden_states = hidden_state_sequence[torch.arange(batch_size), sequence_lengths]


        logits = self.output_layer(final_hidden_states)

        return logits

In [ ]:
def train(model, dataloader, optimizer, device='cpu'):
    model.to(device)
    model.train()
    criterion = nn.BCEWithLogitsLoss()

    total_loss = 0
    total_f1 = 0

    for input_ids, labels in dataloader:
        input_ids, labels = input_ids.to(device), labels.to(device).float()

        optimizer.zero_grad()
        logits = model(input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        preds = (torch.sigmoid(logits) > 0.5).float()
        f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro', zero_division=1)

        total_loss += loss.item()
        total_f1 += f1

    avg_loss = total_loss / len(dataloader)
    avg_f1 = total_f1 / len(dataloader)

    print(f"Train Loss: {avg_loss:.4f}, Train F1: {avg_f1:.4f}")

In [ ]:
def evaluate(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for input_ids, labels in dataloader:
            input_ids, labels = input_ids.to(device), labels.to(device).float()
            logits = model(input_ids)

            preds = (torch.sigmoid(logits) > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)
    return f1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RNN(
    vocab_size=len(dictionary),
    num_classes=len(fixed_label_order),
    pad_token_id=dictionary.token2id['PAD'],
    embedding_dim=128,
    hidden_dim=128
).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(40):
    train(model, train_loader, optimizer, device=device)
    f1 = evaluate(model, test_loader, device=device)
    print(f'Epoch {epoch + 1}, F1: {f1:.4f}')

Train Loss: 0.2845, Train F1: 0.2262
Epoch 1, F1: 0.0219
Train Loss: 0.1742, Train F1: 0.4564
Epoch 2, F1: 0.0212
Train Loss: 0.1714, Train F1: 0.4679
Epoch 3, F1: 0.0207
Train Loss: 0.1682, Train F1: 0.4631
Epoch 4, F1: 0.0204
Train Loss: 0.1641, Train F1: 0.4689
Epoch 5, F1: 0.0284
Train Loss: 0.1592, Train F1: 0.4770
Epoch 6, F1: 0.0351
Train Loss: 0.1536, Train F1: 0.4842
Epoch 7, F1: 0.0457
Train Loss: 0.1494, Train F1: 0.4851
Epoch 8, F1: 0.0436
Train Loss: 0.1410, Train F1: 0.5108
Epoch 9, F1: 0.0579
Train Loss: 0.1325, Train F1: 0.5244
Epoch 10, F1: 0.0610
Train Loss: 0.1239, Train F1: 0.5405
Epoch 11, F1: 0.0665
Train Loss: 0.1148, Train F1: 0.5656
Epoch 12, F1: 0.0817
Train Loss: 0.1058, Train F1: 0.5887
Epoch 13, F1: 0.0847
Train Loss: 0.0973, Train F1: 0.6118
Epoch 14, F1: 0.0998
Train Loss: 0.0881, Train F1: 0.6339
Epoch 15, F1: 0.1196
Train Loss: 0.0799, Train F1: 0.6683
Epoch 16, F1: 0.1333
Train Loss: 0.0729, Train F1: 0.6827
Epoch 17, F1: 0.2102
Train Loss: 0.0661, Tra

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [98]:
import torch.nn as nn
from typing import Tuple

In [99]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [100]:
class LSTMLayer(nn.Module):
    def __init__(self, input_dim=128, hidden_dim=128):
        super(LSTMLayer, self).__init__()

        self.hidden_dim = hidden_dim
        self.input_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.forget_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.output_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.memory_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)

    def forward(self, inputs, initial_states=None):
        batch_size, seq_length, feature_dim = inputs.shape
        if initial_states is None:
            hidden_state = torch.zeros(batch_size, self.hidden_dim).to(inputs.device)
            cell_state = torch.zeros(batch_size, self.hidden_dim).to(inputs.device)
        else:
            hidden_state, cell_state = initial_states

        all_hidden_states = []
        for time_step in range(seq_length):
            current_input = inputs[:, time_step, :]

            combined_state = torch.cat((current_input, hidden_state), dim=1)
            input_activation = torch.sigmoid(self.input_gate(combined_state))
            forget_activation = torch.sigmoid(self.forget_gate(combined_state))
            output_activation = torch.sigmoid(self.output_gate(combined_state))
            memory_activation = torch.tanh(self.memory_gate(combined_state))

            cell_state = input_activation * memory_activation + forget_activation * cell_state
            hidden_state = output_activation * torch.tanh(cell_state)

            all_hidden_states.append(hidden_state.unsqueeze(1))

        all_hidden_states = torch.cat(all_hidden_states, dim=1)
        return all_hidden_states

In [101]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, num_classes, pad_token_id, embedding_dim, hidden_dim, num_layers=1):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        layer_sizes = [embedding_dim] + [hidden_dim] * num_layers
        self.lstm_layers = nn.ModuleList([LSTMLayer(layer_sizes[i], layer_sizes[i+1]) for i in range(num_layers)])
        self.fc_output = nn.Linear(hidden_dim, num_classes)
        self.pad_token_id = pad_token_id
        self.hidden_dim = hidden_dim

    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape
        embedded_inputs = self.embedding(input_ids)

        lstm_output = embedded_inputs
        for lstm_layer in self.lstm_layers:
            lstm_output = lstm_layer(lstm_output)

        sequence_lengths = (input_ids == self.pad_token_id).int().argmax(dim=-1) - 1
        sequence_lengths = sequence_lengths % seq_len

        final_hidden_states = lstm_output[torch.arange(batch_size), sequence_lengths]
        logits = self.fc_output(final_hidden_states)
        return logits

In [102]:
def train_model(model, dataloader, optimizer, device='cpu'):
    model.to(device)
    model.train()

    loss_function = nn.BCEWithLogitsLoss()
    total_loss = 0
    total_f1_score = 0

    for input_ids, labels in dataloader:
        input_ids, labels = input_ids.to(device), labels.to(device).float()

        optimizer.zero_grad()
        output_logits = model(input_ids)

        loss = loss_function(output_logits, labels)
        loss.backward()
        optimizer.step()

        predictions = (torch.sigmoid(output_logits) > 0.5).float()
        f1 = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=1)

        total_loss += loss.item()
        total_f1_score += f1

    avg_loss = total_loss / len(dataloader)
    avg_f1 = total_f1_score / len(dataloader)

    print(f"Train Loss: {avg_loss:.4f}, Train F1 Score: {avg_f1:.4f}")

In [103]:
def evaluate_model(model, dataloader, device='cpu'):
    model.to(device)
    model.eval()

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for input_ids, labels in dataloader:
            input_ids, labels = input_ids.to(device), labels.to(device).float()
            output_logits = model(input_ids)

            predictions = (torch.sigmoid(output_logits) > 0.5).float()
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=1)
    return f1

In [104]:
model = LSTM(
    vocab_size=len(dictionary),
    num_classes=len(fixed_label_order),
    pad_token_id=dictionary.token2id['PAD'],
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [107]:
for epoch in range(50):
    train_model(model, train_loader, optimizer, device=device)
    f1_score_result = evaluate_model(model, test_loader, device=device)
    print(f'Epoch {epoch + 1}, F1 Score: {f1_score_result:.4f}')

Train Loss: 0.0915, Train F1 Score: 0.5677
Epoch 1, F1 Score: 0.1443
Train Loss: 0.0859, Train F1 Score: 0.5906
Epoch 2, F1 Score: 0.1518
Train Loss: 0.0812, Train F1 Score: 0.5982
Epoch 3, F1 Score: 0.1556
Train Loss: 0.0759, Train F1 Score: 0.6153
Epoch 4, F1 Score: 0.1653
Train Loss: 0.0713, Train F1 Score: 0.6365
Epoch 5, F1 Score: 0.1717
Train Loss: 0.0660, Train F1 Score: 0.6599
Epoch 6, F1 Score: 0.1884
Train Loss: 0.0610, Train F1 Score: 0.6929
Epoch 7, F1 Score: 0.2100
Train Loss: 0.0568, Train F1 Score: 0.7122
Epoch 8, F1 Score: 0.2083
Train Loss: 0.0528, Train F1 Score: 0.7372
Epoch 9, F1 Score: 0.2388
Train Loss: 0.0499, Train F1 Score: 0.7533
Epoch 10, F1 Score: 0.2312
Train Loss: 0.0458, Train F1 Score: 0.7710
Epoch 11, F1 Score: 0.2467
Train Loss: 0.0419, Train F1 Score: 0.7905
Epoch 12, F1 Score: 0.2524
Train Loss: 0.0387, Train F1 Score: 0.8112
Epoch 13, F1 Score: 0.2696
Train Loss: 0.0358, Train F1 Score: 0.8136
Epoch 14, F1 Score: 0.2763
Train Loss: 0.0332, Train F1 

__Задание 6 (1 балл).__ В этом задании у вас есть две опции на выбор: добавить __двунаправленность__ для LSTM _или_ добавить __многослойность__. Можно сделать и то, и другое, но дополнительных баллов за это мы не дадим, только бесконечный респект. Обе модификации реализуются довольно просто (буквально 4 строчки кода, если вы аккуратно реализовали модель) и дают примерно одинаковый прирост в качестве. Сделайте выводы: стоит ли увеличивать размер модели в несколько раз?